In [1]:
from google.colab import drive
drive.mount('/content/drive')
import io
import os
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
archive_path = '/content/drive/MyDrive/Colab Notebooks/3113_data_analyze/phrase_counts/study_data'

study_file=os.listdir('/content/drive/MyDrive/Colab Notebooks/3113_data_analyze/phrase_counts/study_data')
print(study_file)
test_file = os.listdir('/content/drive/MyDrive/Colab Notebooks/3113_data_analyze/phrase_counts/test_data')
print(test_file)

['abortion_counts.csv', 'blm_movement_counts.csv', 'islam_counts.csv', 'espionage_counts.csv', 'human_rights_counts.csv', 'china_counts.csv', 'judaism_counts.csv', 'military_spending_counts.csv', 'guns_counts.csv', 'sexual_harrassment_counts.csv', 'oil_counts.csv', 'russia_counts.csv', 'native_americans_counts.csv', 'palestine_counts.csv', 'tech_censorship_counts.csv', 'universities_counts.csv']
['affirmative_action_counts.csv', 'israel_counts.csv', 'church_state_counts.csv', 'climate_change_counts.csv', 'cuba_counts.csv', 'prisons_counts.csv', 'public_finance_counts.csv', 'nuclear_weapons_counts.csv', 'police__counts.csv', 'private_finance_counts.csv', 'yemen_counts.csv', 'venezuela_counts.csv', 'us_immigration_counts.csv']


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/3113_data_analyze/phrase_counts/study_data
for i in range(0, len(study_file)-1):
  globals()['study_data{}'.format(i)] = pd.read_csv(study_file[i], encoding='cp949', header=None)
  globals()['study_data{}'.format(i)] = globals()['study_data{}'.format(i)].drop(1, axis=1)

print(globals()['study_data{}'.format(0)].head)

%cd /content/drive/MyDrive/Colab Notebooks/3113_data_analyze/phrase_counts/test_data
for i in range(0, len(test_file)-1):
  globals()['test_data{}'.format(i)] = pd.read_csv(test_file[i], encoding='cp949', header=None)
  globals()['test_data{}'.format(i)] = globals()['test_data{}'.format(i)].drop(1, axis=1)

/content/drive/MyDrive/Colab Notebooks/3113_data_analyze/phrase_counts/study_data
<bound method NDFrame.head of                        0          2         3                     4   \
0                  PHRASE  aljazeera  alternet  americanconservative   
1                pro life         12        13                    68   
2         abortion rights         97        42                    15   
3           anti abortion         53        54                     4   
4             anti choice          0        54                     0   
5           abortion care          7        13                     0   
6               pregnancy         61        69                     8   
7                    kill          2         5                     1   
8        medical abortion          2         0                     0   
9         family planning         18        15                     0   
10           pro abortion          6         1                     3   
11                 murde

In [4]:
for i in range(0, len(study_file)-1):
  a = globals()['study_data{}'.format(i)][0]
  globals()['st_word{}'.format(i)] = a.values.tolist()
  del globals()['st_word{}'.format(i)][0]
#단어 리스트 
print(globals()['st_word{}'.format(0)])

#언론사 리스트
for i in range(0, len(study_file)-1):
  globals()['media_list{}'.format(i)] = globals()['study_data{}'.format(0)].iloc[0].tolist()
  del globals()['media_list{}'.format(i)][0]
print(media_list0)
print(len(media_list0))
#리스트의 0 값은 PHRASE이므로 삭제해줌.

['pro life', 'abortion rights', 'anti abortion', 'anti choice', 'abortion care', 'pregnancy', 'kill', 'medical abortion', 'family planning', 'pro abortion', 'murder', 'fetus', 'mother', 'reproductive rights', 'infanticide', 'abortion access', 'abortion ban', 'infant', 'reproductive health', 'pro choice', 'abortion law', 'medication abortion', 'contraception', 'fetal heartbeat', 'fetal remains', 'abortion services', 'born alive', 'birth control', 'religious liberty', 'aborted', 'abortion rules', 'abortion providers', 'reproductive justice', 'womens rights', 'dead', 'termination', 'abortion clinic', 'adoption', 'rape or incest', 'maternal', 'abortion restrictions', 'abortion pills', 'unborn babies', 'abortionists', 'fetal abnormalities', 'human life', 'heartbeat bill', 'access to abortion', 'abortion industry', 'right to life']
['aljazeera', 'alternet', 'americanconservative', 'americanspectator', 'ap', 'atlantic', 'bbc', 'breitbart', 'buzzfeed', 'cbs', 'cnn', 'commondreams', 'conversati

In [5]:
#최대 단어 3개 뽑을때 사용할 csv 전처리(object 자료형 변환을 위해 삭제)
for i in range(0, len(study_file)-1):
  globals()['st_d{}'.format(i)]=globals()['study_data{}'.format(i)].drop(0).drop(0,axis=1)
  globals()['st_d{}'.format(i)]=globals()['st_d{}'.format(i)].apply(pd.to_numeric)

# globals()['st_d0'][42]=globals()['st_d0'][43]
globals()['st_d{}'.format(0)]


,2,3,4,5,6,7,8,9,10,11,...,38,39,40,41,42,43,44,45,46,47
1,12,13,68,10,7,16,0,200,48,63,...,22,59,52,26,5,131,13,60,242,1
2,97,42,15,5,43,63,10,54,151,282,...,3,71,4,0,32,211,103,461,585,24
3,53,54,4,0,37,49,17,38,188,208,...,11,107,5,0,27,239,143,487,29,3
4,0,54,0,0,0,1,0,8,4,7,...,1,10,0,1,5,5,38,14,19,0
5,7,13,0,1,2,2,1,25,33,40,...,0,27,1,0,7,52,60,76,64,1
6,61,69,8,3,26,93,17,113,172,306,...,14,112,34,5,29,325,262,487,531,26
7,2,5,1,1,0,3,0,13,3,2,...,17,8,19,1,1,21,6,29,14,0
8,2,0,0,0,1,0,4,4,21,2,...,0,2,0,0,0,3,2,8,9,0
9,18,15,0,1,43,10,3,50,42,66,...,0,8,3,0,27,39,12,72,130,9
10,6,1,3,4,1,10,0,42,24,34,...,1,0,9,9,0,16,3,29,28,2


In [6]:
def kth_largest_number(arr, K):
  unique_nums = list(arr)
  sorted_nums = sorted(unique_nums, reverse=True)
  return sorted_nums[K-1]

In [7]:
#가장 숫자 높은 단어 세개씩만 각 열마다...
for k in range(0, len(study_file)-1):
  data = []
  for i in range(2, len(globals()['media_list{}'.format(k)])+1):
    # data.append(kth_largest_number(globals()['st_d{}'.format(k)][i], 1))
    print(kth_largest_number(globals()['st_d{}'.format(k)][i], 1))
  # globals()['st_top{}'.format(k)] = pd.DataFrame(data, columns=['x','y','z'], index=globals()['media_list{}'.format(k)])

# print(globals()['st_top{}'.format(0)])
print(len(data))

97
69
68
10
43
93
17
200
188
306
379
218
86
33
562
216
52
56
40
442
608
410
75
51
64
51
152
776
307
373
122
261
18
256
164
254
22
112
52
26
32
325
262
487
585
632
140
25
19
448
79
30
208
765
262
575
67
435
12
1198
275
29
68
327
184
2114
238
19
73
50
1267
629
72
495
98
166
64
9
32
26
132
5
58
565
65
485
449
329
6
283
542
18
20
28
54
40
62
368
84
21
43
321
74
145
41
74
18
234
89
22
185
485
56
49
59
71
92
26
32
169
201
88
111
10
481
92
16
17
130
37
65
23
64
21
28
60
34
34
40
13
83
60
20
53
28
100
128
100
160
36
41
94
137
46
48
41
178
188
33
131
72
85
43
133
160
68
63
34
75
52
12
156
83
36
81
82
33
56
277
34
588
26
39
94
25
27
121
285
65
47
98
59
299
296
75
278
166
82
26
138
64
135
48
348
130
558
21
36
58
59
68
114
175
126
21
168
92
69
25
30
79
119
46
41
51
797
334
219
73
51
21
196
195
77
167
51
668
225
224
44
2275
170
4550
377
116
486
333
751
427
705
310
789
44
47
211
15
54
1505
203
1620
101
301
162
91
12
171
198
168
2
697
179
92
24
20
16
57
121
49
267
26
159
25
194
616
156
38
66
701
207


KeyError: ignored

In [ ]:
#딕셔너리 테스트 
for k in range(0, len(study_file)-1):
  data = {}
  for i in range(2, len(globals()['media_list{}'.format(k)])+1):
    data[globals()['media_list{}'.format(k)][i-2]]=[kth_largest_number(globals()['st_d{}'.format(k)][i],1),\
            kth_largest_number(globals()['st_d{}'.format(k)][i],2),\
            kth_largest_number(globals()['st_d{}'.format(k)][i],3)]
  globals()['st_top{}'.format(k)] = pd.DataFrame(data, columns=['x','y','z'], index=globals()['media_list{}'.format(k)])

# print(globals()['st_top{}'.format(0)])
print(data[0])

In [ ]:
!git init 

In [ ]:
!git add .

In [ ]:
!git commit -m '1st'

In [ ]:
!git push -u origin main

In [ ]:
!git remote rm origin